In [1]:
import numpy as np 
import matplotlib.pyplot as plt
import glob
import cv2
from keras.models import Model, Sequential
from keras.layers import Dense, Flatten, Conv2D, MaxPooling2D
from keras.layers.normalization import BatchNormalization
import os
import seaborn as sns
from keras.applications.vgg16 import VGG16
from keras.applications.xception import Xception
from keras.applications.inception_v3 import InceptionV3
from sklearn import preprocessing
from sklearn.ensemble import RandomForestClassifier
from google.colab import drive
import xgboost as xgb
from sklearn.metrics import confusion_matrix
from sklearn import metrics

## loading the dataset

In [2]:
SIZE = 256
train_images = []
train_labels = [] 

### importing each image and set the current directory name as its label 

In [3]:
ext = ['png', 'jpg', 'jpeg'] 

for directory_path in glob.glob("PATH TO YOU LOCAL DATASET"):
    splited = directory_path.split("/") #use \ if you're on windows 
    label = splited[-1]
    print(label)
    for e in ext:
      for img_path in glob.glob(os.path.join(directory_path, "*." + e)):
          #print(img_path)
           img = cv2.imread(img_path, cv2.IMREAD_COLOR)       
           img = cv2.resize(img, (SIZE, SIZE))
           train_images.append(img)
           train_labels.append(label)

Pneumonia
No_findings
Covid-19


In [4]:
train_images = np.array(train_images)
train_labels = np.array(train_labels)

In [5]:
test_images = []
test_labels = [] 

In [6]:
ext = ['png', 'jpg', 'jpeg'] 

for directory_path in glob.glob("PATH TO YOU LOCAL DATASET"):
    splited_test = directory_path.split("/") #use \ if you're on windows
    label_test = splited_test[-1]
    print(label_test)
    for e in ext:
      for img_path in glob.glob(os.path.join(directory_path, "*." + e)):
          #print(img_path)
          img = cv2.imread(img_path, cv2.IMREAD_COLOR)       
          img = cv2.resize(img, (SIZE, SIZE))
          test_images.append(img)
          test_labels.append(label_test)

Pneumonia
No_findings
Covid-19


In [7]:
test_images = np.array(test_images)
test_labels = np.array(test_labels)

## Pre-processing

### label encoding

In [8]:
le = preprocessing.LabelEncoder()
le.fit(test_labels)
test_labels_encoded = le.transform(test_labels)
le.fit(train_labels)
train_labels_encoded = le.transform(train_labels)

### spliting the dataset into training set and test set
we already split the dataset in our directory, but for the matter of clean coding, we're changing their name.

In [9]:
x_train, y_train, x_test, y_test = train_images, train_labels_encoded, test_images, test_labels_encoded

### scaling train set and test set

In [10]:
x_train, x_test = x_train / 255.0, x_test / 255.0

## Feature Extraction

### VGG16

In [11]:
features = []
extracted_feature = []

In [25]:
VGG_model = VGG16(weights='imagenet', include_top=False, input_shape=(SIZE, SIZE, 3), pooling='avg')

58900480/58889256 [==============================] - 0s 0us/step


In [26]:
extracted_feature = VGG_model.predict(x_train)

In [27]:
extracted_feature.shape

(900, 512)

In [ ]:
features = extracted_feature.reshape(extracted_feature.shape[0], -1)

### Inception

In [14]:
features = []
extracted_feature = []

In [15]:
inception_model = InceptionV3(weights='imagenet', include_top=False, input_shape=(SIZE, SIZE, 3), pooling='avg')

87924736/87910968 [==============================] - 1s 0us/step


In [16]:
extracted_feature =  inception_model.predict(x_train)

In [17]:
extracted_feature.shape

(900, 2048)

In [18]:
features = extracted_feature.reshape(extracted_feature.shape[0], -1)

## Xception

In [46]:
features = []
extracted_feature = []

In [12]:
xception_model = Xception(weights='imagenet', include_top=False, input_shape=(SIZE, SIZE, 3),)

In [13]:
extracted_feature =  xception_model.predict(x_train)

In [14]:
features = extracted_feature.reshape(extracted_feature.shape[0], -1)

## traning the model
### using extracted features to train the xgbclassifier

In [15]:
model = xgb.XGBClassifier(objective='multi:softmax')
model.fit(features, y_train) #For sklearn no one hot encoding

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0,
              learning_rate=0.1, max_delta_step=0, max_depth=3,
              min_child_weight=1, missing=None, n_estimators=100, n_jobs=1,
              nthread=None, objective='multi:softprob', random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
              silent=None, subsample=1, verbosity=1)

## predict test set

In [16]:
X_test_feature = xception_model.predict(x_test)
X_test_features = X_test_feature.reshape(X_test_feature.shape[0], -1)

In [17]:
prediction = model.predict(X_test_features)
#Inverse le transform to get original label back. 
prediction = le.inverse_transform(prediction)

## accuracy score

In [18]:
print ("Accuracy = ", metrics.accuracy_score(test_labels, prediction))

Accuracy =  0.7333333333333333


## confusion matrix

In [ ]:
cm = confusion_matrix(test_labels, prediction)

In [ ]:
sns.heatmap(cm, annot=True)
plt.show()

In [ ]:
n=np.random.randint(0, x_test.shape[0])
img = x_test[n]
plt.imshow(img)
input_img = np.expand_dims(img, axis=0) #Expand dims so the input is (num images, x, y, c)
input_img_feature=VGG_model.predict(input_img)
input_img_features=input_img_feature.reshape(input_img_feature.shape[0], -1)
prediction = model.predict(input_img_features)[0] 
prediction = le.inverse_transform([prediction])  #Reverse the label encoder to original name
print("The prediction for this image is: ", prediction)
print("The actual label for this image is: ", test_labels[n])